# 🖥️ Esperimento MNIST: Bayes by Backprop con tutte le combinazioni di π, σ₁, σ₂ + Baseline SGD e SGD con Dropout
#
In questo notebook addestriamo:
1. Il modello Bayes by Backprop su MNIST per ogni combinazione di (π, σ₁, σ₂).  
2. Una rete feedforward standard (due hidden layer) con ottimizzazione SGD senza regularizzazione.  
3. La stessa rete feedforward, ma con Dropout sui due hidden layer, sempre ottimizzata con SGD.  
#
In ogni ciclo di training mostriamo barre di avanzamento sia per le epoche che per i batch, insieme ai print dei principali metriche.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import trange, tqdm
import math

# Impostiamo device (GPU se disponibile)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"🚀 Usando device: {device}")

🚀 Usando device: cuda


## 📦 Caricamento e preprocessing del dataset MNIST

In [2]:
# %% [code]
# 📦 TensorFlow + TF Probability
import tensorflow as tf

# Caricamento dataset
(x_train_tf, y_train_tf), (x_test_tf, y_test_tf) = tf.keras.datasets.mnist.load_data()

# Normalizzazione e reshape
x_train_tf = x_train_tf.astype('float32') / 255.0
x_test_tf  = x_test_tf.astype('float32') / 255.0
x_train_tf = x_train_tf.reshape(-1, 28 * 28)
x_test_tf  = x_test_tf.reshape(-1, 28 * 28)

print(f"✅ TF MNIST: {x_train_tf.shape[0]} train, {x_test_tf.shape[0]} test, input_dim = {x_train_tf.shape[1]}")

# %% [code]
# 🔥 PyTorch
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Trasformazione: ToTensor + flatten
transform = transforms.Compose([
    transforms.ToTensor(),                              # [0,255]→[0,1], C×H×W→FloatTensor
    transforms.Lambda(lambda x: x.view(-1))             # 28×28 → vettore 784
])

# Download e dataset
train_dataset_torch = datasets.MNIST(root='./data', train=True,  download=True, transform=transform)
test_dataset_torch  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader
BATCH_SIZE_TORCH = 512
train_loader_torch = DataLoader(train_dataset_torch, batch_size=BATCH_SIZE_TORCH,
                                shuffle=True,  num_workers=4, pin_memory=True)
test_loader_torch  = DataLoader(test_dataset_torch,  batch_size=1000,
                                shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ Torch MNIST: {len(train_dataset_torch)} train, {len(test_dataset_torch)} test, input_dim = {28*28}")

# Iperparametri
NUM_EPOCHS = 600
LR = 1e-3
BATCH_SIZE = 1024
INPUT_DIM = 28 * 28
HIDDEN_DIM = 400
OUTPUT_DIM = 10
INPUT_DIM = 28 * 28
N_TRAIN = len(train_dataset_torch)
N_TEST  = len(test_dataset_torch)
print(f"✅ Caricati MNIST: {N_TRAIN} train, {N_TEST} test, input_dim = {INPUT_DIM}")

2025-06-07 10:31:36.326954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749292296.503649      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749292296.555963      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
✅ TF MNIST: 60000 train, 10000 test, input_dim = 784


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.63MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 148kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.41MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.56MB/s]

✅ Torch MNIST: 60000 train, 10000 test, input_dim = 784
✅ Caricati MNIST: 60000 train, 10000 test, input_dim = 784


## 🔧 Definizione dei modelli
#
1. **BayesianLinear** e **BayesianNet** (Bayes by Backprop)  
2. **FFNet** (Feedforward standard) e **FFNetDropout** (stessa architettura + Dropout)

In [3]:
# ------------------------------------------------------------
# (1) Layer BayesianLinear e modello BayesianNet
# ------------------------------------------------------------
class BayesianLinear(nn.Module):
    """
    Layer fully-connected bayesiano:
     - parametri μ e ρ per pesi e bias
     - σ = log(1 + exp(ρ))
     - log_qw calcolato internamente ad ogni forward
    """
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.in_features  = in_features
        self.out_features = out_features

        # Parametri μ e ρ
        self.mu_weight  = nn.Parameter(torch.zeros(out_features, in_features))
        self.rho_weight = nn.Parameter(torch.ones(out_features, in_features) * -3.0)
        self.mu_bias    = nn.Parameter(torch.zeros(out_features))
        self.rho_bias   = nn.Parameter(torch.ones(out_features) * -3.0)

        # log_qw verrà memorizzato ad ogni campionamento
        self.log_qw = None

    def compute_log_qw(self, w: torch.Tensor, sigma_w: torch.Tensor) -> torch.Tensor:
        """
        Calcola log q(w) = -0.5 * sum( log(2π σ_w^2) + ((w - μ_w)^2) / σ_w^2 )
        """
        return -0.5 * torch.sum(torch.log(2 * torch.pi * sigma_w**2) +
                                ((w - self.mu_weight) ** 2) / (sigma_w**2))

    def sample_weight_and_bias(self) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        """
        Campiona (w, b) e restituisce anche log_qw per il calcolo del KL.
        """
        sigma_w = torch.log1p(torch.exp(self.rho_weight))
        sigma_b = torch.log1p(torch.exp(self.rho_bias))

        eps_w = torch.randn_like(self.mu_weight)
        eps_b = torch.randn_like(self.mu_bias)
        w = self.mu_weight + sigma_w * eps_w
        b = self.mu_bias   + sigma_b * eps_b

        log_qw = self.compute_log_qw(w, sigma_w)
        return w, b, log_qw

    def forward(self, x: torch.Tensor, sample: bool = True) -> torch.Tensor:
        """
        Se sample=True e modello in training, campiona w e b. Altrimenti usa μ.
        Calcola internamente log_qw e restituisce F.linear(x, w, b).
        """
        sigma_w = torch.log1p(torch.exp(self.rho_weight))
        sigma_b = torch.log1p(torch.exp(self.rho_bias))

        if self.training and sample:
            eps_w = torch.randn_like(self.mu_weight)
            eps_b = torch.randn_like(self.mu_bias)
            w = self.mu_weight + sigma_w * eps_w
            b = self.mu_bias   + sigma_b * eps_b
        else:
            w = self.mu_weight
            b = self.mu_bias

        # Calcolo log_qw (usando w campionato o µ se sample=False, ma in ogni caso sufficiente)
        self.log_qw = -0.5 * torch.sum(torch.log(2 * torch.pi * sigma_w**2) +
                                       ((w - self.mu_weight)**2) / (sigma_w**2))

        return F.linear(x, w, b)


def log_mixture_prior(w_flat: torch.Tensor, pi: float, sigma1: float, sigma2: float) -> torch.Tensor:
    """
    Calcola log [ π N(0,σ1^2) + (1−π) N(0,σ2^2) ] per ogni elemento di w_flat e somma tutti i log-prob.
    """
    # 1) costanti di log(2π σ^2) come scalari
    log_const1 = 0.5 * math.log(2 * math.pi * sigma1**2)
    log_const2 = 0.5 * math.log(2 * math.pi * sigma2**2)

    # 2) log-densità per ogni elemento
    log_prob1 = -0.5 * (w_flat ** 2) / (sigma1 ** 2) - log_const1
    log_prob2 = -0.5 * (w_flat ** 2) / (sigma2 ** 2) - log_const2

    # 3) log(π) e log(1−π) come scalari
    log_pi         = math.log(pi)
    log_1_minus_pi = math.log(1.0 - pi)

    # 4) termini per log-sum-exp
    t1 = log_prob1 + log_pi
    t2 = log_prob2 + log_1_minus_pi

    # 5) log-sum-exp stabile
    max_log = torch.max(t1, t2)
    log_mix = max_log + torch.log(torch.exp(t1 - max_log) + torch.exp(t2 - max_log))

    # 6) somma finale
    return log_mix.sum()


class BayesianNet(nn.Module):
    """
    Rete bayesiana fully-connected a 3 strati (2 hidden + output).
    La dimensione dei layer hidden è hidden_dim.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, pi, sigma1, sigma2):
        super().__init__()
        # Parametri di prior
        self.pi     = pi
        self.sigma1 = sigma1
        self.sigma2 = sigma2

        # Tre layer bayesiani
        self.bl1 = BayesianLinear(input_dim,  hidden_dim)
        self.bl2 = BayesianLinear(hidden_dim, hidden_dim)
        self.bl3 = BayesianLinear(hidden_dim, output_dim)

        # Accumulatore di KL
        self.kl = None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Azzero contatore KL
        self.kl = 0.0

        #######  LAYER 1  #######
        w1, b1, log_qw1 = self.bl1.sample_weight_and_bias()
        w1_flat = torch.cat([w1.view(-1), b1.view(-1)])
        log_pw1 = log_mixture_prior(w1_flat, self.pi, self.sigma1, self.sigma2)
        self.kl = self.kl + (log_qw1 - log_pw1)
        x1 = F.linear(x, w1, b1)
        x1 = F.relu(x1)

        #######  LAYER 2  #######
        w2, b2, log_qw2 = self.bl2.sample_weight_and_bias()
        w2_flat = torch.cat([w2.view(-1), b2.view(-1)])
        log_pw2 = log_mixture_prior(w2_flat, self.pi, self.sigma1, self.sigma2)
        self.kl = self.kl + (log_qw2 - log_pw2)
        x2 = F.linear(x1, w2, b2)
        x2 = F.relu(x2)

        #######  LAYER 3 (OUTPUT)  #######
        w3, b3, log_qw3 = self.bl3.sample_weight_and_bias()
        w3_flat = torch.cat([w3.view(-1), b3.view(-1)])
        log_pw3 = log_mixture_prior(w3_flat, self.pi, self.sigma1, self.sigma2)
        self.kl = self.kl + (log_qw3 - log_pw3)
        logits = F.linear(x2, w3, b3)

        return logits

train_loader = DataLoader(
    train_dataset_torch,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset_torch,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)
class FFNet(nn.Module):
    """
    Rete feedforward standard con due hidden layer ReLU e output layer senza Dropout.
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim,  hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class FFNetDropout(nn.Module):
    """
    Stessa architettura di FFNet, ma con Dropout p=0.5 sui due hidden layer.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, p: float = 0.5):
        super().__init__()
        self.fc1 = nn.Linear(input_dim,  hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

criterion = nn.CrossEntropyLoss()

## 🎯 Iperparametri di training

In [4]:
NUM_EPOCHS_BBB = 600
LR_BBB = 1e-3
PI_LIST = [0.75]
SIGMA1_LIST = [np.exp(-2)]
SIGMA2_LIST = [np.exp(-6)]

# Per i modelli SGD baseline (nr di epoche e lr allineati a BBB)
NUM_EPOCHS_SGD = 20
LR_SGD = 1e-3

criterion = nn.CrossEntropyLoss()

print("ℹ️ Iperparametri:")
print(f"  Bayes by Backprop: NUM_EPOCHS = {NUM_EPOCHS_BBB}, LR = {LR_BBB}")
print(f"    PI_LIST     = {PI_LIST}")
print(f"    SIGMA1_LIST = {SIGMA1_LIST}")
print(f"    SIGMA2_LIST = {SIGMA2_LIST}")
print(f"\n  Baseline SGD: NUM_EPOCHS = {NUM_EPOCHS_SGD}, LR = {LR_SGD}\n")

ℹ️ Iperparametri:
  Bayes by Backprop: NUM_EPOCHS = 600, LR = 0.001
    PI_LIST     = [0.75]
    SIGMA1_LIST = [0.1353352832366127]
    SIGMA2_LIST = [0.0024787521766663585]

  Baseline SGD: NUM_EPOCHS = 20, LR = 0.001



## SGD

In [5]:
# %% [code]
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

# ---- Copia delle funzioni dal primo notebook ----

# 1️⃣ Costruzione dei modelli
def build_sgd_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)  # logits
    ])

def build_dropout_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10)
    ])

# 2️⃣ Funzione di training
def train_model(model, name, x_train, y_train, epochs=10):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    print(f"\n🔧 Training {name}")
    model.fit(x_train, y_train,
              batch_size=128,
              epochs=epochs,
              validation_split=0.1,
              verbose=2)
    print(f"✅ Finished training {name}")
    return model


# Allenamento SGD puro
sgd_model = train_model(build_sgd_model(), "SGD", x_train_tf, y_train_tf, epochs=NUM_EPOCHS_SGD)

# %% [code]
# %% [code]
# Allenamento SGD + Dropout
dropout_model = train_model(build_dropout_model(), "Dropout", x_train_tf, y_train_tf, epochs=NUM_EPOCHS_SGD)
loss_sgd, acc_sgd = sgd_model.evaluate(x_test_tf, y_test_tf, verbose=0)
loss_do,  acc_do  = dropout_model.evaluate(x_test_tf, y_test_tf, verbose=0)

print(f"📈 SGD senza regularizzazione Accuracy: {acc_sgd:.4f}")
print(f"📈 SGD + Dropout (p=0.5) Accuracy: {acc_do:.4f}")

I0000 00:00:1749292325.003349      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



🔧 Training SGD
Epoch 1/20


I0000 00:00:1749292326.984369      60 service.cc:148] XLA service 0x7ac0b4003060 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749292326.984837      60 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1749292327.258803      60 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1749292328.118870      60 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


422/422 - 5s - 11ms/step - accuracy: 0.8984 - loss: 0.3518 - val_accuracy: 0.9610 - val_loss: 0.1459
Epoch 2/20
422/422 - 1s - 2ms/step - accuracy: 0.9583 - loss: 0.1454 - val_accuracy: 0.9717 - val_loss: 0.1050
Epoch 3/20
422/422 - 1s - 2ms/step - accuracy: 0.9686 - loss: 0.1016 - val_accuracy: 0.9712 - val_loss: 0.1041
Epoch 4/20
422/422 - 1s - 2ms/step - accuracy: 0.9768 - loss: 0.0783 - val_accuracy: 0.9778 - val_loss: 0.0819
Epoch 5/20
422/422 - 1s - 2ms/step - accuracy: 0.9815 - loss: 0.0609 - val_accuracy: 0.9750 - val_loss: 0.0866
Epoch 6/20
422/422 - 1s - 3ms/step - accuracy: 0.9851 - loss: 0.0488 - val_accuracy: 0.9763 - val_loss: 0.0892
Epoch 7/20
422/422 - 1s - 3ms/step - accuracy: 0.9878 - loss: 0.0401 - val_accuracy: 0.9770 - val_loss: 0.0878
Epoch 8/20
422/422 - 1s - 3ms/step - accuracy: 0.9907 - loss: 0.0317 - val_accuracy: 0.9762 - val_loss: 0.0888
Epoch 9/20
422/422 - 1s - 2ms/step - accuracy: 0.9923 - loss: 0.0262 - val_accuracy: 0.9778 - val_loss: 0.0856
Epoch 10/20

## SGD con Dropout

In [6]:
# ------------------------------------------------------------
# 🎯 Allenamento: SGD con Dropout p=0.5
# ------------------------------------------------------------
'''model_do = FFNetDropout(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, p=0.5).to(device)
optimizer_do = optim.SGD(model_do.parameters(), lr=LR)
best_test_acc_do = 0.0

for epoch in trange(NUM_EPOCHS, desc="SGD + Dropout", leave=True):
    model_do.train()
    running_loss = 0.0
    total_train = 0
    correct_train = 0
    num_batches = len(train_loader)

    # --- Fase di training ---
    for data, target in tqdm(train_loader, desc=f"Epoca {epoch+1}/{NUM_EPOCHS}", leave=False):
        data = data.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)

        optimizer_do.zero_grad()
        output = model_do(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer_do.step()

        running_loss += loss.item()
        preds = output.argmax(dim=1)
        correct_train += (preds == target).sum().item()
        total_train += target.size(0)

    train_acc = correct_train / total_train * 100

    # --- Valutazione su test (Dropout disabilitato in eval) ---
    model_do.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)
            logits = model_do(data)
            preds = logits.argmax(dim=1)
            correct_test += (preds == target).sum().item()
            total_test += target.size(0)

    test_acc = correct_test / total_test * 100
    if test_acc > best_test_acc_do:
        best_test_acc_do = test_acc

    # Stampa delle metriche a fine epoca
    print(
        f"Epoch {epoch+1:03d}/{NUM_EPOCHS} | "
        f"Train Loss = {running_loss/num_batches:.4f} | "
        f"Train Acc = {train_acc:.2f}% | "
        f"Test Acc = {test_acc:.2f}%"
    )

print(f"\n✅ SGD + Dropout (p=0.5) → Best Test Acc = {best_test_acc_do:.2f}%")'''

'model_do = FFNetDropout(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, p=0.5).to(device)\noptimizer_do = optim.SGD(model_do.parameters(), lr=LR)\nbest_test_acc_do = 0.0\n\nfor epoch in trange(NUM_EPOCHS, desc="SGD + Dropout", leave=True):\n    model_do.train()\n    running_loss = 0.0\n    total_train = 0\n    correct_train = 0\n    num_batches = len(train_loader)\n\n    # --- Fase di training ---\n    for data, target in tqdm(train_loader, desc=f"Epoca {epoch+1}/{NUM_EPOCHS}", leave=False):\n        data = data.to(device, non_blocking=True)\n        target = target.to(device, non_blocking=True)\n\n        optimizer_do.zero_grad()\n        output = model_do(data)\n        loss = criterion(output, target)\n        loss.backward()\n        optimizer_do.step()\n\n        running_loss += loss.item()\n        preds = output.argmax(dim=1)\n        correct_train += (preds == target).sum().item()\n        total_train += target.size(0)\n\n    train_acc = correct_train / total_train * 100\n\n    # --- Valut

## 🔄 Allenamento Bayes by Backprop per ogni combinazione di (π, σ₁, σ₂)
Qui usiamo barre di avanzamento con tqdm sia per le epoche che per i batch.

In [7]:
results_bbb = []  # memorizza (pi, sigma1, sigma2, best_test_acc)

for pi in PI_LIST:
    for sigma1 in SIGMA1_LIST:
        for sigma2 in SIGMA2_LIST:
            model = BayesianNet(INPUT_DIM, hidden_dim=400, output_dim=10,
                                pi=pi, sigma1=sigma1, sigma2=sigma2).to(device)
            optimizer = optim.Adam(model.parameters(), lr=LR_BBB)
            best_test_acc = 0.0

            # Barra di avanzamento per le epoche
            epoch_desc = f"BBB π={pi:.2f} σ₁={sigma1:.1e} σ₂={sigma2:.1e}"
            for epoch in trange(NUM_EPOCHS_BBB, desc=epoch_desc, leave=True):
                model.train()
                running_loss = 0.0
                total = 0
                correct = 0
                num_batches = len(train_loader)

                # Training loop senza tqdm sui batch
                for data, target in train_loader:
                    data, target = data.to(device), target.to(device)
                    optimizer.zero_grad()
                    output = model(data)                              # forward bayesiano
                    nll = criterion(output, target)                   # negative log-likelihood
                    kl_loss = model.kl / N_TRAIN                      # KL normalizzato
                    loss = nll + kl_loss
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item()
                    preds = output.argmax(dim=1)
                    correct += (preds == target).sum().item()
                    total += target.size(0)

                train_acc = correct / total * 100

                # Valutazione su test (usando i pesi μ, sample=False)
                model.eval()
                correct_test = 0
                total_test = 0
                with torch.no_grad():
                    for data, target in test_loader:
                        data, target = data.to(device), target.to(device)
                        x = F.relu(model.bl1(data, sample=False))
                        x = F.relu(model.bl2(x, sample=False))
                        logits = model.bl3(x, sample=False)
                        preds = logits.argmax(dim=1)
                        correct_test += (preds == target).sum().item()
                        total_test += target.size(0)
                test_acc = correct_test / total_test * 100
                if test_acc > best_test_acc:
                    best_test_acc = test_acc

                # Print delle metriche a fine epoca
                print(
                    f"  Epoca {epoch+1:02d}/{NUM_EPOCHS_BBB} | "
                    f"Train Loss={running_loss/num_batches:.4f} | "
                    f"Train Acc={train_acc:.2f}% | "
                    f"Test Acc={test_acc:.2f}%"
                )

            print(f"✅ BBB π={pi:.2f}, σ₁={sigma1:.1e}, σ₂={sigma2:.1e} → "
                  f"Best Test Acc = {best_test_acc:.2f}%")
            results_bbb.append((pi, sigma1, sigma2, best_test_acc))


BBB π=0.75 σ₁=1.4e-01 σ₂=2.5e-03:   0%|          | 1/600 [00:04<48:25,  4.85s/it]

  Epoca 01/600 | Train Loss=7.4942 | Train Acc=10.27% | Test Acc=17.64%


BBB π=0.75 σ₁=1.4e-01 σ₂=2.5e-03:   0%|          | 2/600 [00:08<41:12,  4.13s/it]

  Epoca 02/600 | Train Loss=7.3670 | Train Acc=12.12% | Test Acc=31.57%


BBB π=0.75 σ₁=1.4e-01 σ₂=2.5e-03:   0%|          | 3/600 [00:12<39:03,  3.93s/it]

...

  Epoca 598/600 | Train Loss=2.0714 | Train Acc=96.19% | Test Acc=97.71%


BBB π=0.75 σ₁=1.4e-01 σ₂=2.5e-03: 100%|█████████▉| 599/600 [37:06<00:03,  3.68s/it]

  Epoca 599/600 | Train Loss=2.0765 | Train Acc=96.15% | Test Acc=97.47%


BBB π=0.75 σ₁=1.4e-01 σ₂=2.5e-03: 100%|██████████| 600/600 [37:09<00:00,  3.72s/it]

  Epoca 600/600 | Train Loss=2.0757 | Train Acc=96.16% | Test Acc=97.29%
✅ BBB π=0.75, σ₁=1.4e-01, σ₂=2.5e-03 → Best Test Acc = 97.83%


## 📋 Risultati Aggregati

In [8]:
print("\n📊 Risultati Bayes by Backprop (π, σ₁, σ₂, Best Test Accuracy):")
for pi, s1, s2, acc in results_bbb:
    print(f"  π={pi:.2f}, σ₁={s1:.4e}, σ₂={s2:.4e}  →  Best Test Acc = {acc:.2f}%")

acc_baseline_sgd = acc_sgd * 100
acc_baseline_do  = acc_do  * 100

print("\n📊 Risultati Baseline TF:")
print(f"  SGD senza regularizzazione       → Best Test Acc = {acc_baseline_sgd:.2f}%")
print(f"  SGD + Dropout (p=0.5)             → Best Test Acc = {acc_baseline_do:.2f}%")



📊 Risultati Bayes by Backprop (π, σ₁, σ₂, Best Test Accuracy):
  π=0.75, σ₁=1.3534e-01, σ₂=2.4788e-03  →  Best Test Acc = 97.83%

📊 Risultati Baseline TF:
  SGD senza regularizzazione       → Best Test Acc = 97.90%
  SGD + Dropout (p=0.5)             → Best Test Acc = 97.14%
